In [1]:
import os

from tqdm import tqdm
from PIL import Image
import numpy as np

In [2]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

In [15]:
os.chdir('../')
from data.base_dataset import Normalize_image
from utils.saving_utils import load_checkpoint

In [5]:
from model import U2NET

In [36]:
device = "cuda"
image_dir = "/Users/narekgeghamyan/Classes/MLE_bootcamp/Capstone_Project/input_images"
result_dir = "/Users/narekgeghamyan/Classes/MLE_bootcamp/Capstone_Project/output_images"
# checkpoint_path = "/Users/narekgeghamyan/Classes/MLE_bootcamp/Capstone_Project/results/training_cloth_segm_u2net_exp1/checkpoints/itr_00001000_u2net.pth"
checkpoint_path = "/Users/narekgeghamyan/Classes/MLE_bootcamp/Capstone_Project/results/cloth_segm_u2net_latest.pth"
do_palette = True

In [8]:
def get_palette(num_cls):
    """Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= ((lab >> 0) & 1) << (7 - i)
            palette[j * 3 + 1] |= ((lab >> 1) & 1) << (7 - i)
            palette[j * 3 + 2] |= ((lab >> 2) & 1) << (7 - i)
            i += 1
            lab >>= 3
    return palette

In [9]:
transforms_list = []
transforms_list += [transforms.ToTensor()]
transforms_list += [Normalize_image(0.5, 0.5)]
transform_rgb = transforms.Compose(transforms_list)

In [37]:
net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)
# net = net.to(device)
# net = net.eval()

palette = get_palette(4)

----checkpoints loaded from path: /Users/narekgeghamyan/Classes/MLE_bootcamp/Capstone_Project/results/cloth_segm_u2net_latest.pth----


In [38]:
images_list = sorted(os.listdir(image_dir))
pbar = tqdm(total=len(images_list))
for image_name in images_list:
    img = Image.open(os.path.join(image_dir, image_name)).convert("RGB")
    image_tensor = transform_rgb(img)
    image_tensor = torch.unsqueeze(image_tensor, 0)

    output_tensor = net(image_tensor) #net(image_tensor.to(device))
    output_tensor = F.log_softmax(output_tensor[0], dim=1)
    output_tensor = torch.max(output_tensor, dim=1, keepdim=True)[1]
    output_tensor = torch.squeeze(output_tensor, dim=0)
    output_tensor = torch.squeeze(output_tensor, dim=0)
    output_arr = output_tensor.cpu().numpy()

    output_img = Image.fromarray(output_arr.astype("uint8"), mode="L")
    if do_palette:
        output_img.putpalette(palette)
    output_img.save(os.path.join(result_dir, image_name[:-3] + "png"))

    pbar.update(1)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [4:21:28<00:00, 5229.37s/it]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:51<00:00, 15.14s/it]